In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from datetime import date, timedelta, datetime
import numpy as np
import matplotlib.pyplot as plt

from urllib.request import urlopen
import json


In [ ]:
def get_colors(df, colors, cat_options):
    
    scheme = []
    categories = df.unique()

    if len(categories) > len(colors):
        print("Error in method get_colors: More categories than color options")
        print("Exiting...\n")
        exit()
        
    if len(cat_options) != len(colors):
        print("Error in method get_colors: Categories options doesn't match color options")
        print("Exiting...\n")
        exit()
        
    for i in range(len(cat_options)):
        if cat_options[i] in categories:
            scheme.append(colors[i])
    
    return scheme

In [ ]:
with urlopen('https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json') as response:
    state_fips = json.load(response)


In [ ]:
ces = pd.read_csv("ces_after.csv", dtype={"Fips":str})

In [ ]:
ces.loc[ces.Positivity < 0,"PosTestCategory"] = "Unknown"
ces.loc[(ces.Positivity >= 0) & (ces.Positivity < 5),"PosTestCategory"] = "Low"
ces.loc[(ces.Positivity >= 5) & (ces.Positivity < 10),"PosTestCategory"] = "Moderate"
ces.loc[(ces.Positivity >= 10) &(ces.Positivity < 20),"PosTestCategory"] = "High"
ces.loc[(ces.Positivity >= 20),"PosTestCategory"] = "Extreme"


ces.sort_values(by="Positivity", ascending=False, inplace=True)
# ces["ReportDate"] = ces["ReportDate"].dt.strftime("%Y-%m-%d")
dt = ces["ReportDate"].max()
dt = datetime.strptime(dt, "%Y-%m-%d")
month = datetime.strptime(str(dt.month), "%m")
month = month.strftime("%B")
fulldate = f"{month} {dt.day}, {dt.year}"

In [ ]:
ces

In [ ]:
color_scheme = ["#d55e00", "#cc72b2", "#f0e442", "#009e73", "#f0f0f0"]
cat_options = ["Extreme", "High", "Moderate", "Low", "Unknown"]
# color_scheme = ["#cc72b2", "#f0e442", "#009e73", "#f0f0f0"]
# red, pink,  yellow, green, gray


color_scheme = get_colors(ces["PosTestCategory"], color_scheme, cat_options)
print(color_scheme)

In [ ]:
fig = px.choropleth_mapbox(ces, geojson=state_fips, locations="Fips", color="PosTestCategory",
                           color_discrete_sequence=color_scheme,
                           mapbox_style="carto-positron",
#                            mapbox_style = "albers usa",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           labels={'Positivity':'Positive Test Rate (%) ',
                                   'PosTestCategory' : 'Category Ranking',
                                   'ReportDate' : 'Report Date',
                                   'PositivityIs' : '2-Week Positivity Trend is'},
                           hover_name="State",
                           hover_data={'Fips':False,
                                       'State':False,
                                       'ReportDate':True,
                                       'PositivityIs':True,
                                       'Positivity':': .1f',
                                       'PosTestCategory':False,
                            }
                          )

fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0}, title=f"Positivity by State ({fulldate})", legend_orientation="h")

fig.show()

In [ ]:
fig.write_html(f"./state_positivity_2020-09-09.html")